In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
import keras

In [5]:
data = pd.read_csv('data/data12.csv')

In [6]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state = 2 )

In [7]:
X_train = train_set.drop(columns=['Y'], axis=1).values 
y_train = train_set.drop(columns=['X1','X2','X3'], axis=1).values
X_test = test_set.drop(columns=['Y'], axis=1).values
y_test = test_set.drop(columns=['X1','X2','X3'], axis=1).values

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold

In [8]:
from sklearn.model_selection import GridSearchCV
xgb_model  = xgb.XGBRegressor() 
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}
grid_search = GridSearchCV (xgb_model, param_grid,cv=5, scoring = 'neg_mean_squared_error' )
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [9]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}

In [14]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5) 
R2_scores_vali =[]
R2_scores_train5f =[]
valiY_1 =[]
trainY_1 =[]
ypre_in_vali_1 =[]
ypre_in_train_1  =[]
for train_indices, test_indices in kf.split(X_train): 
    trainX, valiX = X_train[train_indices], X_train[test_indices] 
    trainY, valiY = y_train[train_indices], y_train[test_indices]
    xgb_model = xgb.XGBRegressor(learning_rate = 0.1, max_depth = 3, n_estimators = 200)
    model = xgb_model.fit(X_train,y_train)
    ypre_in_vali = xgb_model.predict(valiX)
    ypre_in_train = xgb_model.predict(trainX)
    R2_in_vali = r2_score(valiY , ypre_in_vali)
    R2_in_train5f = r2_score(trainY , ypre_in_train)
    R2_scores_vali.append(R2_in_vali) 
    R2_scores_train5f.append(R2_in_train5f) 
    valiY_1.append(valiY)
    trainY_1.append(trainY)
    ypre_in_vali_1.append(ypre_in_vali)
    ypre_in_train_1.append(ypre_in_train)
    print (" 5fcv_xgboost_R2_scores_vali：",  R2_scores_vali)
    print ("5fcv_xgboost_R2_scores_train：",  R2_scores_train5f)
    print ("valiY：",  valiY_1)
    print ("trainY：",  trainY_1)
    print ("vali_prey：",  ypre_in_vali_1)
    print ("train_prey：",  ypre_in_train_1)

 5fcv_xgboost_R2_scores_vali： [0.999327518241363]
5fcv_xgboost_R2_scores_train： [0.998957323380104]
valiY： [array([[22086.19355],
       [18298.98039],
       [18064.     ],
       [21234.16216],
       [20836.09023],
       [13578.66667]])]
trainY： [array([[21092.     ],
       [20784.50526],
       [16995.90244],
       [17454.54545],
       [22574.54545],
       [22640.64   ],
       [20924.14458],
       [15599.48387],
       [17929.3945 ],
       [17911.56044],
       [20854.51852],
       [21590.44156],
       [22061.41935],
       [18070.85714],
       [15645.53846],
       [17342.35897],
       [22086.62069],
       [15488.     ],
       [21610.14634],
       [21210.83871],
       [15058.28571],
       [21421.55932]])]
vali_prey： [array([22124.35 , 18298.07 , 17951.596, 21136.062, 20750.207, 13618.933],
      dtype=float32)]
train_prey： [array([21057.758, 20762.412, 17087.803, 17614.885, 22543.18 , 22614.389,
       20893.46 , 15467.379, 17969.467, 17906.729, 20990.557, 21579.0

In [15]:
xgb_model = xgb.XGBRegressor(learning_rate = 0.1, max_depth= 3, n_estimators= 200)
model = xgb_model.fit(X_train,y_train)
testpre =  xgb_model.predict(X_test)
mse_in_test = mean_squared_error (y_test , testpre)
rmse_in_test = np.sqrt (mse_in_test)
R2_in_test = r2_score(y_test , testpre)

import joblib as joblib
joblib.dump(xgb_model, "pima2.joblib.dat")
print ("xgb_R2_train：", R2_in_test)
print ("testY：",  y_test)
print ("test_preY：",  testpre)

xgb_R2_train： 0.8282782255023825
testY： [[22249.73913]
 [20924.08163]
 [22260.36364]
 [13957.26027]
 [13388.25532]
 [21714.48889]
 [13333.94286]
 [23244.8    ]]
test_preY： [21604.47  21344.268 22228.902 16420.318 16270.586 21994.176 16271.578
 22768.479]


# Transfer learning combined XGB

In [17]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os

new_data = pd.read_csv('data/data12.csv')
new_train_set, new_test_set = train_test_split(new_data, test_size=0.2, random_state= 2 )
X1_train = new_train_set.drop(columns=['Y'], axis=1).values
y1_train = new_train_set['Y'].values
X1_test = new_test_set.drop(columns=['Y'], axis=1).values
y1_test = new_test_set['Y'].values
sc = StandardScaler()
X1_train = sc.fit_transform(X1_train)
X1_test = sc.transform(X1_test)
xgb_model = joblib.load("pimadachang.joblib.dat")
train_pred_orig = xgb_model.predict(X1_train)
test_pred_orig = xgb_model.predict(X1_test)
X1_train_new = np.hstack((X1_train, train_pred_orig.reshape(-1, 1)))
X1_test_new = np.hstack((X1_test, test_pred_orig.reshape(-1, 1)))
xgb_model_new = xgb.XGBRegressor()
xgb_model_new.fit(X1_train_new, y1_train)
joblib.dump(xgb_model_new, "transferred_model.joblib.dat")
# model after transfer learning
train_pre_new = xgb_model_new.predict(X1_train_new)
rmse_in_train_new = np.sqrt(mean_squared_error(y1_train, train_pre_new))
R2_in_train_new = r2_score(y1_train, train_pre_new)
print("TL_R2_training：", R2_in_train_new)
test_pre_new = xgb_model_new.predict(X1_test_new)
R2_in_test_new = r2_score(y1_test, test_pre_new)
print("TL_R2_test：", R2_in_test_new)

new_data_to_predict = pd.read_csv('screening0607.csv')
X_new = new_data_to_predict.values
X_new = sc.transform(X_new)

new_pred_orig = xgb_model.predict(X_new)

X_new_with_pred = np.hstack((X_new, new_pred_orig.reshape(-1, 1)))

new_predictions = xgb_model_new.predict(X_new_with_pred)


new_data_to_predict['Predicted_Y'] = new_predictions
output_folder = 'Predictions'
os.makedirs(output_folder, exist_ok=True)
new_data_to_predict.to_csv(os.path.join(output_folder, 'predictions74.csv'), index=False)
print("The predicted results have been saved to the specified folder.")


[15:38:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

TL_R2_training： 0.999999999998141
TL_R2_test： 0.9856237595972309
The predicted results have been saved to the specified folder.
